In [11]:
def read_input(filename):
    rides = []
    
    with open(filename) as pizza_file:
        for i, line in enumerate(pizza_file):
            if i == 0:
                R, C, F, N, B, T = [int(e) for e in line.strip().split(' ')]
            else:
                a, b, x, y, s, f = [int(e) for e in line.strip().split(' ')]
                rides.append({
                    'start': (a, b),
                    'finish': (x, y),
                    'earliest_start': s,
                    'latest_finish': f
                })
    return {
        'rows': R,
        'columns': C,
        'no_vehicles': F,
        'no_rides': N,
        'bonus': B,
        'no_steps': T,
        'rides': rides
    }

In [140]:
import time
class ProgressPrinter:
    def __init__(self, iterations, filename):
        self.iterations = iterations
        self.start = time.time()
        self.i = 0
        self.logfile = open(filename.replace('.in', '.log'), 'a')

    def print_progress(self, text=''):
        now = time.time()
        average = (now - self.start) / (self.i + 1)
        should_end = average * (self.iterations - self.i)
        total = average * self.iterations
        self.logfile.write("{} -- {}/{}. Average {:.2f}. Remaining {:.2f}. So far {:.2f}. Total? {:.2f}\n".format(
            text, self.i, self.iterations, average, should_end, now - self.start, total))
        self.i += 1

In [141]:
class Position:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return "Position({}, {})".format(self.x, self.y)
    
    def __str__(self):
        return self.__repr__()
    
    def distance(self, position):
        return abs(self.x - position.x) + abs(self.y - position.y)
    
    
class Ride:
    def __init__(self, start, finish, earliest, latest, pk):
        self.start = start
        self.finish = finish
        self.earliest = earliest
        self.latest = latest
        self.done = False
        self.assigned = False
        self.id = pk
        self.distance = self.start.distance(self.finish)
        
    def __repr__(self):
        return "Ride({}, {}, {}, {})".format(self.start, self.finish, self.earliest, self.latest)
    
    def __str__(self):
        return self.__repr__()
    
    
class World:
    def __init__(self, rows, columns, vehicles, bonus, rides, steps):
        self.rows = rows
        self.columns = columns
        self.vehicles = vehicles
        self.bonus = bonus
        self.rides = rides
        self.steps = steps
        
    def __repr__(self):
        return "World({}, {}, {}, {}, {}, {})".format(
            self.rows, self.columns, self.vehicles, self.bonus, self.rides, self.steps
        )
    
    def __str__(self):
        return self.__repr__()

class Car:
    def __init__(self, position=Position(0, 0)):
        self.position=position
        self.rides = []
        self.current_ride = None
        self.remaining_steps = None
    
    def find_nearest_ride(self, rides, step):
        min_dist = float('Inf')
        ride_id = None
        for ride in rides.values():
            ride_dist = self.position.distance(ride.start) + max(0, ride.earliest - step)
            if (ride.latest - step) < ride_dist:
                # skip rides with no price
                continue
            
            if ride_dist < min_dist:
                min_dist = ride_dist
                ride_id = ride.id
        return ride_id
    
    def get_remaining_steps(self):
        self.remaining_steps = self.position.distance(self.current_ride.start) + self.position.distance(self.current_ride.finish)
    
    def decrement_steps(self):
        self.remaining_steps -= 1
        
        if self.remaining_steps == 0:
            self.rides.append(self.current_ride.id)
            self.position = self.current_ride.finish
            self.current_ride.done = True
            self.current_ride = None
            self.remaining_steps = None
    
    def __repr__(self):
        return "Car({})".format(self.rides)
    
    def __str__(self):
        return self.__repr__()
        
class Simulation:
    def __init__(self, filename):
        self.filename = filename
        self.world = self.read_world(filename)
        self.cars = []
        for i in range(self.world.vehicles):
            self.cars.append(Car())
        self.pp = ProgressPrinter(self.world.steps, filename)
            
    def read_world(self, filename):
        a = read_input(filename)
        rides = {}
        for i, r in enumerate(a['rides']):
            ride = Ride(
                start=Position(*r['start']),
                finish=Position(*r['finish']),
                earliest=r['earliest_start'],
                latest=r['latest_finish'],
                pk=i
            )
            rides[i] = ride
        world = World(
            rows=a['rows'],
            columns=a['columns'],
            vehicles=a['no_vehicles'],
            bonus=a['bonus'],
            rides=rides,
            steps=a['no_steps']
        )
        return world
    
    def perform_step(self, step):
        self.pp.print_progress()
        unassigned_rides = {r.id: r for r in self.world.rides.values() if not r.assigned or r.latest < step}
        free_cars = [c for c in self.cars if not c.current_ride]
        
        for car in free_cars:
            next_ride_id = car.find_nearest_ride(unassigned_rides, step)
            if next_ride_id is None:
                continue
            
            next_ride = self.world.rides[next_ride_id]
            next_ride.assigned = True
            car.current_ride = next_ride
            car.get_remaining_steps()
            unassigned_rides.pop(next_ride_id)
            
        occupied_cars = [c for c in self.cars if c.current_ride]
        
        for car in occupied_cars:
            if step < car.current_ride.earliest and car.remaining_steps > car.current_ride.distance:
                continue
            car.decrement_steps()
            
    
    def process_world(self):
        for i in xrange(self.world.steps):
            self.perform_step(i+1)
            
def get_output(s):
    lines = []
    for i, car in enumerate(s.cars, start=1):
        lines.append("{} {}".format(i, ' '.join([str(r) for r in car.rides])))
    return lines

def write_output(s):
    with open(s.filename.replace('.in', '.out'), 'w') as f:
        f.write('\n'.join(get_output(s)))

In [139]:
s = Simulation('a_example.in')
s.process_world()
s.cars
write_output(s)

In [142]:
s = Simulation('b_should_be_easy.in')
s.process_world()
s.cars

[Car([0, 184, 144, 269, 208]),
 Car([167, 54, 150, 73, 178]),
 Car([296, 87, 220, 32, 106]),
 Car([216, 8, 225, 23, 194]),
 Car([180, 4, 198, 287]),
 Car([65, 88, 261, 101, 138, 2]),
 Car([33, 210, 267, 42, 257]),
 Car([108, 92, 53, 16]),
 Car([116, 134, 44, 93, 149]),
 Car([85, 56, 22, 272]),
 Car([164, 246, 232, 291]),
 Car([249, 187, 114, 278]),
 Car([75, 57, 10, 247]),
 Car([103, 193, 191, 251]),
 Car([72, 125, 76, 169]),
 Car([7, 38, 235]),
 Car([80, 19, 70, 151]),
 Car([212, 263, 140, 49, 211]),
 Car([294, 226, 45]),
 Car([148, 245, 5]),
 Car([161, 64, 12, 215]),
 Car([127, 258, 237, 143]),
 Car([259, 40, 155]),
 Car([13, 254, 104]),
 Car([133, 253, 262]),
 Car([162, 172, 266]),
 Car([62, 268, 100]),
 Car([203, 243, 17]),
 Car([159, 102, 55]),
 Car([136, 185, 182]),
 Car([260, 197, 94]),
 Car([95, 117, 195, 229]),
 Car([58, 298, 97]),
 Car([276, 109, 233, 270]),
 Car([290, 166, 124]),
 Car([227, 48, 153]),
 Car([273, 190, 52]),
 Car([137, 86, 43, 231]),
 Car([218, 147, 173]),
 Ca

In [112]:
write_output(s)

In [ ]:
s = Simulation('c_no_hurry.in')
s.process_world()
s.cars
write_output(s)
s.cars